<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_data_import_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Import Pipeline

This code is still under construction, and is therefore very very bad in places. 

### TODO
- Import CO2 dataset
- Extract unnecessary methods into normal python files and import where necessary. 
- Remove unnecessary variable changes where necessary - this stacks up all the JSON, making everything harder than it needs to be. 
  - Actually, it might not
- Change unnecessary image imports to feature imports 
- Figure out a way of iterating through existing images and displaying the area currently covered by my dataset on a map. 
- Define and import other regions of interest - stick to cities and suburbs for now, since that will have the best health data. Converting this to include rural or rocky areas is an increase in feature set. 
- Figure out how accurate the image exports are
  - Are the points definitely centered on the given coordinates? 
  - is there a way of standardising lighting? 
- add file indexing into one CSV with all our existing latlong exports, so we're not constantly querying the filesystem. 
- list exported files into a CSV 
- Combine GHG exports into one CSV 

## Setup
*   Import necessary libraries
*   Import fyputil module
*   Set up Earth Engine authentication and mount google drive  


In [1]:
import ee
# TODO test not importing this and still use eec.map 
import folium
import os

from google.colab import drive
from osgeo import gdal
from PIL import Image
from pprint import pprint

In [ ]:
ee.Authenticate()
ee.Initialize()
drive.mount('/content/drive')

In [3]:
%cd /content
%rm -rf fyp

/content


In [ ]:
# Import FYP repo so we can access fyputil common library 
%cd /content
!git clone https://github.com/WRFitch/fyp.git

In [ ]:
# Import fyputil library
%cd fyp/src/fyputil
import constants as c
import ee_constants as eec
import ee_utils as eeutil
import fyp_utils as fyputil
%cd /content

# Dataset import

### Import the following datasets into Google Drive

*   [Sentinel-2 Satellite photography](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
*   [Sentinel-5 Precursor Data](https://developers.google.com/earth-engine/datasets/catalog/sentinel)
  *   [Carbon Monoxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CO)
  *   [Formaldehyde](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_HCHO)
  *   [Nitrogen Dioxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_NO2)
  *   [Ozone](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_O3)
  *   [Sulphur Dioxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_SO2)
  *   [Methane](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CH4)
*   [ODIAC Fossil Fuel CO2 Emissions](https://db.cger.nies.go.jp/dataset/ODIAC/DL_odiac2019.html)

##### Import necessary variables

In [6]:
# Define dataset boundaries for britain and london 
great_britain = eec.great_britain
london = eec.london
uxbridge = eec.uxbridge
millennium_dome = eec.millennium_dome
greenwich = eec.greenwich
w_hemisphere = eec.west_hemisphere
e_hemisphere = eec.east_hemisphere

In [11]:
# TODO move this into a CONSTANTS.py file
# Earth engine username, used to import classified image into ee assets folder
USERNAME = 'wrfitch'
OUTPUT_DIR = USERNAME + "/out/"

# Define collections for each dataset to be used 
s2 = eec.s2
s5_CO = eec.s5_CO
s5_HCHO = eec.s5_HCHO
s5_NO2 = eec.s5_NO2
s5_O3 = eec.s5_O3
s5_SO2 = eec.s5_SO2
s5_CH4 = eec.s5_CH4
#TODO import CO2 dataset

CO_band = c.CO_band
HCHO_band = c.HCHO_band
NO2_band = c.NO2_band
O3_band = c.O3_band
SO2_band = c.SO2_band
CH4_band = c.CH4_band

start_date = eec.start_date
end_date = eec.end_date
vis_palette = eec.vis_palette

drive_path = c.drive_path
export_dir = c.export_dir
geotiff_dir = c.geotiff_dir

In [13]:
# Import datasets 
# TODO analyse whether it makes sense to analyse these on a highly localised level
# TODO figure out whether this object duplication is an antipattern that will 
#      eat all my RAM 
s2_img = eec.s2_img
CO_img = eec.CO_img
HCHO_img = eec.HCHO_img
NO2_img = eec.NO2_img
O3_img = eec.O3_img
SO2_img = eec.SO2_img
CH4_img = eec.CH4_img

s2_id = eec.s2_id
CO_id = eec.CO_id
HCHO_id = eec.HCHO_id
NO2_id = eec.NO2_id
O3_id = eec.O3_id
SO2_id = eec.SO2_id
CH4_id = eec.CH4_id

# For easier iteration down the line. I know I'm not supposed to, but google 
# can't tell me what to do, even if it's a good idea!
ghg_imgs = [CO_img, HCHO_img, NO2_img, O3_img, SO2_img, CH4_img]
ghg_ids = [CO_id, HCHO_id, NO2_id, O3_id, SO2_id, CH4_id]

### Visualise Data

In [8]:
eec.map 

### Export Data

Exports as .csv tables and GeoTIFF images. 

#### Exporting CSVs

This method of getting the data is very very stupid, but also it does exactly what I need. 

In [ ]:
# Only once this is completed can you move forward and get pictures from these spreadsheets.
for ghg_img in ghg_imgs:
  csv_name = ghg_img.getInfo().get('bands')[0].get('id')
  #exportTableFromImage(ghg_img,london, 1000, export_dir, csv_name)

#### Getting Images From CSV Data

In [ ]:
eeutil.getImgsFromCsv(f"{c.data_dir}/{c.SO2_band}.csv", s2_img)

In [ ]:
#pprint(ee.batch.Task.list())

# Data processing

In [ ]:
fyputil.geotiffToPng(geotiff_dir, rm_artifacts=False)
fyputil.moveFilesByExtension(geotiff_dir, f"{export_dir}/geotiff", ".tif")
fyputil.moveFilesByExtension(geotiff_dir, f"{export_dir}/png", ".png")
fyputil.rmConversionArtifacts(geotiff_dir, rmTif=False, rmXml=True)

In [ ]:
# Cleaning up if things go a bit wrong
fyputil.moveFilesByExtension(f"{export_dir}/geotiff", f"{export_dir}/png", ".png")
fyputil.rmConversionArtifacts(f"{export_dir}/geotiff", rmTif=False, rmXml=True)
fyputil.moveFilesByExtension(geotiff_dir, f"{export_dir}/geotiff", ".tif")

In [23]:
# TODO add these all into one csv file as part of import pipeline. 
fyputil.parseCsvCoords(f"{drive_path}{export_dir}/{CO_band}.csv")
fyputil.parseCsvCoords(f"{drive_path}{export_dir}/{HCHO_band}.csv")
fyputil.parseCsvCoords(f"{drive_path}{export_dir}/{NO2_band}.csv")
fyputil.parseCsvCoords(f"{drive_path}{export_dir}/{O3_band}.csv")
fyputil.parseCsvCoords(f"{drive_path}{export_dir}/{SO2_band}.csv")
fyputil.parseCsvCoords(f"{drive_path}{export_dir}/{CH4_band}.csv")

/content/drive/MyDrive/img_export/CO_column_number_density.csv
/content/drive/MyDrive/img_export/CO_column_number_density.csv has already been processed - exiting...
/content/drive/MyDrive/img_export/tropospheric_HCHO_column_number_density.csv
/content/drive/MyDrive/img_export/tropospheric_HCHO_column_number_density.csv has already been processed - exiting...
/content/drive/MyDrive/img_export/tropospheric_NO2_column_number_density.csv
/content/drive/MyDrive/img_export/tropospheric_NO2_column_number_density.csv has already been processed - exiting...
/content/drive/MyDrive/img_export/O3_column_number_density.csv
/content/drive/MyDrive/img_export/O3_column_number_density.csv has already been processed - exiting...
/content/drive/MyDrive/img_export/SO2_column_number_density.csv
/content/drive/MyDrive/img_export/SO2_column_number_density.csv has already been processed - exiting...
/content/drive/MyDrive/img_export/CH4_column_volume_mixing_ratio_dry_air.csv
/content/drive/MyDrive/img_export